In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from tqdm import tqdm
import ipywidgets as widgets
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import re
from selenium.webdriver.support.ui import Select

In [4]:
import pandas as pd
import numpy as np

In [12]:
options = webdriver.ChromeOptions()

prefs = {"download.default_directory" : "/home/lim/dev_ws/eda/data", "download.prompt_for_download":False}
options.add_experimental_option("prefs", prefs)

url = "https://www.kobis.or.kr/kobis/business/stat/boxs/findYearlyBoxOfficeList.do"
driver = webdriver.Chrome(service=Service("/home/lim/dev_ws/eda/driver/chromedriver"), options=options)

driver.get(url)
driver.set_window_size(1800, 1300)

In [13]:
year = driver.find_element(By.ID, "sSearchYearFrom")
years = []
years_list = year.find_elements(By.TAG_NAME, "option")
for yearly in years_list:
    yearly = yearly.get_attribute("value")
    years.append(yearly)
years

['2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022',
 '2023',
 '2024']

In [14]:
driver.close()

In [7]:
year.send_keys(years[-1])

In [8]:
movie_names = []
page = driver.page_source
soup = BeautifulSoup(page, "html.parser")
movies = soup.find("table", "tbl_comm th_sort")
moviesList = movies.find_all("span", class_="ellip per90")
for movie in moviesList:
    movie = movie.find("a").text
    movie_names.append(movie)

movie_names

['파묘',
 '범죄도시4',
 '인사이드 아웃 2',
 '베테랑2',
 '파일럿',
 '웡카',
 '모아나 2',
 '소방관',
 '탈주',
 '하얼빈',
 '위키드',
 '듄: 파트2',
 '에이리언: 로물루스',
 '데드풀과 울버린',
 '핸섬가이즈',
 '하이재킹',
 '쿵푸팬더4',
 '베놈: 라스트 댄스',
 '시민덕희',
 '퓨리오사: 매드맥스 사가',
 '슈퍼배드 4',
 '외계+인 2부',
 '위시',
 '서울의 봄',
 '사랑의 하츄핑',
 '그녀가 죽었다',
 '건국전쟁',
 '노량: 죽음의 바다',
 '히든페이스',
 '댓글부대',
 '글래디에이터 Ⅱ',
 '혹성탈출: 새로운 시대',
 '대도시의 사랑법',
 '청설',
 '명탐정 코난: 100만 달러의 펜타그램',
 '극장판 하이큐!! 쓰레기장의 결전',
 '행복의 나라',
 '탈출: 프로젝트 사일런스',
 '와일드 로봇',
 '보통의 가족',
 '원더랜드',
 '트위스터스',
 '무파사: 라이온 킹',
 '조커: 폴리 아 되',
 '아마존 활명수',
 '콰이어트 플레이스: 첫째 날',
 '극장판 짱구는 못말려: 우리들의 공룡일기',
 '설계자',
 '고질라 X 콩: 뉴 엠파이어',
 '빅토리']

In [9]:
movie_viewers = []
movieL = driver.find_elements(By.ID, "td_audiAcc")
for movie in movieL:
    movie_viewers.append(movie.text)

movie_viewers

['11,914,766',
 '11,502,779',
 '8,799,013',
 '7,525,339',
 '4,718,036',
 '3,531,717',
 '3,338,348',
 '3,216,504',
 '2,561,872',
 '2,385,334',
 '2,029,548',
 '2,019,129',
 '2,002,033',
 '1,977,010',
 '1,777,008',
 '1,774,150',
 '1,771,738',
 '1,763,723',
 '1,715,064',
 '1,606,378',
 '1,562,001',
 '1,430,121',
 '1,403,486',
 '1,273,640',
 '1,239,133',
 '1,237,103',
 '1,173,927',
 '1,135,516',
 '1,011,002',
 '972,991',
 '909,792',
 '907,155',
 '879,109',
 '801,756',
 '753,606',
 '744,539',
 '710,620',
 '689,603',
 '682,367',
 '650,336',
 '625,908',
 '620,240',
 '614,754',
 '612,831',
 '606,612',
 '552,619',
 '532,932',
 '525,369',
 '510,370',
 '508,467']

In [22]:
movie_genre = []
for movie in tqdm(movie_names):
    link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, movie)))
    link.click()

    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")

    for dt in soup.find_all("dt", text="요약정보"):
        partDets = []
        dd = dt.find_next("dd")  # Get the associated <dd> element
        if dd:
            # Extract and clean the genre information
            raw_text = dd.get_text(separator="|").strip()  # Use '|' as separator for clarity
            parts = raw_text.split("|")  # Split by '|'
            genres = [
                '미스터리', '공포(호러)', '액션', '범죄', '판타지', '드라마', 'SF', '다큐멘터리',
                '코미디', '뮤지컬', '애니메이션', '스릴러', '어드벤처', '사극', '멜로/로맨스', '기타'
            ]
            for part in parts:
                part = part.strip()
                items = re.split((","), part)
                for item in items:
                    item = item.strip()
                    if item in genres :
                        partDets.append(item)
            movie_genre.append(partDets)

    close = driver.find_element(By.CLASS_NAME, "close")
    close.click()

movie_genre, len(movie_genre)

  0%|          | 0/50 [00:00<?, ?it/s]/tmp/ipykernel_4841/4026965249.py:9: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for dt in soup.find_all("dt", text="요약정보"):
100%|██████████| 50/50 [01:39<00:00,  2.00s/it]


([['미스터리', '공포(호러)'],
  ['액션', '범죄'],
  ['애니메이션'],
  ['액션', '범죄'],
  ['코미디'],
  ['판타지', '드라마'],
  ['애니메이션'],
  ['드라마'],
  ['액션'],
  ['액션'],
  ['공포(호러)', 'SF'],
  ['액션', '코미디', 'SF'],
  ['판타지', '뮤지컬'],
  ['코미디'],
  ['범죄', '액션'],
  ['애니메이션', '액션', '코미디'],
  ['액션', 'SF', '스릴러'],
  ['드라마'],
  ['액션'],
  ['애니메이션', '액션', '어드벤처', '코미디'],
  ['드라마'],
  ['기타'],
  ['애니메이션'],
  ['드라마'],
  ['애니메이션'],
  ['미스터리', '스릴러'],
  ['다큐멘터리'],
  ['사극', '액션'],
  ['스릴러'],
  ['범죄', '드라마'],
  ['액션', '사극'],
  ['액션', 'SF'],
  ['드라마'],
  ['드라마', '멜로/로맨스'],
  ['애니메이션'],
  ['애니메이션'],
  ['드라마'],
  ['스릴러'],
  ['애니메이션'],
  ['드라마'],
  ['드라마'],
  ['액션', '어드벤처', '드라마'],
  ['범죄', '드라마', '뮤지컬'],
  ['코미디'],
  ['스릴러'],
  ['범죄', '드라마'],
  ['액션', '어드벤처', 'SF'],
  ['드라마'],
  ['애니메이션'],
  ['애니메이션']],
 50)

In [ ]:
movie_names = []
movie_viewers = []
movie_genre = []
movie_years = []
movieG = []
years = []


options = webdriver.ChromeOptions()

prefs = {"download.default_directory" : "/home/lim/dev_ws/eda/data", "download.prompt_for_download":False}
options.add_experimental_option("prefs", prefs)

url = "https://www.kobis.or.kr/kobis/business/stat/boxs/findYearlyBoxOfficeList.do"
driver = webdriver.Chrome(service=Service("/home/lim/dev_ws/eda/driver/chromedriver"), options=options)

driver.get(url)
driver.set_window_size(1800, 1300)

year_dropdown = driver.find_element(By.ID, "sSearchYearFrom")
select = Select(year_dropdown)  # Use Select to interact with the dropdown
years_list = select.options  # Get all options in the dropdown


for option in years_list:
    years.append(option.get_attribute("value"))

# Limit to the last few years (example: from the 10th year onward)
def dataCrawling(yearly):
    year_dropdown = driver.find_element(By.ID, "sSearchYearFrom")
    select = Select(year_dropdown)  # Re-initialize the Select object
    
    driver.execute_script("window.scrollTo(0, 0)")
    select.select_by_value(yearly)  # Select the year from the dropdown
    
    # Click the "show" button to load the data
    show = driver.find_element(By.CLASS_NAME, "wrap_btn")
    show.click()
    time.sleep(1)

    movieN = []
    # Wait for the page to load and parse it
    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")
    movies = soup.find("table", "tbl_comm th_sort")
    moviesList = movies.find_all("span", class_="ellip per90")
    for movie in moviesList:
        movie = movie.find("a").text
        movie_names.append(movie)
        movieN.append(movie)
        movie_years.append(yearly)
    
    movieL = driver.find_elements(By.ID, "td_audiAcc")
    for movie in movieL:
        movie_viewers.append(movie.text)

    for movie in tqdm(movieN):
        movieG = []
        link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, movie)))
        link.click()

        page = driver.page_source
        soup = BeautifulSoup(page, "html.parser")

        for dt in soup.find_all("dt", text="요약정보"):
            partDets = []
            dd = dt.find_next("dd")  # Get the associated <dd> element
            if dd:
                # Extract and clean the genre information
                raw_text = dd.get_text(separator="|").strip()  # Use '|' as separator for clarity
                parts = raw_text.split("|")  # Split by '|'
                genres = [
                    '미스터리', '공포(호러)', '액션', '범죄', '판타지', '드라마', 'SF', '다큐멘터리', '전쟁',
                    '코미디', '뮤지컬', '애니메이션', '스릴러', '어드벤처', '사극', '멜로/로맨스', '기타'
                ]
                for part in parts:
                    part = part.strip()
                    items = re.split((","), part)
                    for item in items:
                        item = item.strip()
                        if item in genres :
                            partDets.append(item)
                movie_genre.append(partDets)

        close = driver.find_element(By.CLASS_NAME, "close")
        close.click()
        
    driver.close()

(0, 0, 0, 0)

In [75]:
len(movie_names), len(movie_viewers), len(movie_genre), len(movie_years)

(100, 100, 100, 100)

In [34]:
movie_genre, len(movie_genre)

([['드라마'],
  ['판타지', '드라마'],
  ['액션'],
  ['액션', '어드벤처', 'SF'],
  ['범죄', '액션'],
  ['액션', '드라마'],
  ['액션'],
  ['범죄', '드라마'],
  ['멜로/로맨스', '판타지', '뮤지컬'],
  ['액션', '어드벤처', '코미디'],
  ['액션', '어드벤처', '판타지', 'SF'],
  ['범죄'],
  ['액션', '드라마'],
  ['사극', '드라마'],
  ['액션', '어드벤처', '판타지'],
  ['액션', '범죄', '스릴러'],
  ['애니메이션', '드라마', '멜로/로맨스'],
  ['애니메이션'],
  ['코미디'],
  ['액션', '어드벤처', '판타지', '코미디'],
  ['범죄', '액션'],
  ['액션', '드라마', '스릴러', '전쟁'],
  ['액션', 'SF'],
  ['범죄', '스릴러'],
  ['액션', '어드벤처', 'SF'],
  ['범죄', '코미디'],
  ['범죄', '액션'],
  ['애니메이션', '코미디'],
  ['드라마'],
  ['드라마'],
  ['애니메이션'],
  ['액션', '범죄'],
  ['액션', 'SF'],
  ['액션', '어드벤처', '판타지', 'SF'],
  ['공포(호러)', '미스터리', '스릴러', '드라마'],
  ['액션', '어드벤처', '드라마', 'SF'],
  ['드라마'],
  ['공포(호러)'],
  ['다큐멘터리'],
  ['액션', '어드벤처', '판타지', 'SF'],
  ['액션', '코미디'],
  ['액션', '어드벤처', '판타지'],
  ['스릴러'],
  ['코미디', '어드벤처', '액션'],
  ['코미디'],
  ['미스터리', '스릴러'],
  ['공포(호러)', '미스터리', '스릴러'],
  ['범죄', '드라마'],
  ['드라마'],
  ['SF', '스릴러'],
  ['판타지', '드라마'],
  ['액션', '어드벤처', '판타지', '

In [22]:
movie_names = []
movie_viewers = []
movie_genre = []
movie_years = []
movieG = []

options = webdriver.ChromeOptions()

prefs = {"download.default_directory" : "/home/lim/dev_ws/eda/data", "download.prompt_for_download":False}
options.add_experimental_option("prefs", prefs)

url = "https://www.kobis.or.kr/kobis/business/stat/boxs/findYearlyBoxOfficeList.do"
driver = webdriver.Chrome(service=Service("/home/lim/dev_ws/eda/driver/chromedriver"), options=options)

driver.get(url)
driver.set_window_size(1800, 1300)
time.sleep(1)

year_dropdown = driver.find_element(By.ID, "sSearchYearFrom")
select = Select(year_dropdown)  # Use Select to interact with the dropdown
years_list = select.options  # Get all options in the dropdown

years = []
for option in years_list:
    years.append(option.get_attribute("value"))

# Limit to the last few years (example: from the 10th year onward)
# Loop through the years and perform scraping
for yearly in tqdm(years):
    dataCrawling(yearly)

len(movie_names), len(movie_viewers), len(movie_genre), len(movie_years)

  0%|          | 0/21 [00:00<?, ?it/s]


TypeError: dataCrawling() takes 0 positional arguments but 1 was given

In [12]:
movie_genre

[['코미디', '드라마'],
 ['SF', '액션'],
 ['사극', '드라마'],
 ['액션', '어드벤처', 'SF'],
 ['액션'],
 ['액션', '코미디', '드라마'],
 ['드라마'],
 ['스릴러'],
 ['범죄', '기타'],
 ['범죄', '액션'],
 ['드라마', '스릴러', 'SF', '액션', '어드벤처'],
 ['범죄', '액션'],
 ['코미디'],
 ['액션'],
 ['SF', '드라마'],
 ['드라마'],
 ['드라마', '어드벤처', '액션'],
 ['액션', '어드벤처', '판타지'],
 ['액션', '코미디', '범죄'],
 ['액션'],
 ['멜로/로맨스', '코미디'],
 ['범죄', '액션', '스릴러'],
 ['드라마'],
 ['드라마'],
 ['액션', 'SF', '어드벤처'],
 ['액션', '스릴러'],
 ['공포(호러)', '스릴러'],
 ['SF'],
 ['액션', '스릴러'],
 ['어드벤처', '드라마', '판타지'],
 ['스릴러'],
 ['애니메이션'],
 ['멜로/로맨스', '코미디'],
 ['액션', '어드벤처', 'SF'],
 ['액션'],
 ['스릴러', '드라마'],
 ['드라마'],
 ['드라마', '액션'],
 ['드라마'],
 ['액션', 'SF', '어드벤처'],
 ['어드벤처', '드라마'],
 ['액션', 'SF'],
 ['드라마', '멜로/로맨스'],
 ['액션', '범죄', '스릴러'],
 ['액션', '코미디'],
 ['코미디', '드라마'],
 ['멜로/로맨스', '코미디'],
 ['드라마', '액션'],
 ['SF', '액션'],
 ['공포(호러)', '스릴러'],
 ['사극', '액션'],
 ['애니메이션', '어드벤처', '코미디', '뮤지컬', '판타지'],
 ['SF'],
 ['어드벤처'],
 ['드라마'],
 ['드라마'],
 ['드라마'],
 ['액션', 'SF', '어드벤처'],
 ['사극', '액션'],
 ['액션', 'SF'],
 ['액션', '판타지

In [10]:
data = {"Name" : movie_names, "Year" : movie_years, "Genre" : movie_genre, "Viewers" : movie_viewers}
df = pd.DataFrame(data)
df.head()

,Name,Year,Genre,Viewers
0,7번방의 선물,2013,"[코미디, 드라마]","12,811,206"
1,설국열차,2013,"[SF, 액션]","9,349,993"
2,관상,2013,"[사극, 드라마]","9,134,238"
3,아이언맨 3,2013,"[액션, 어드벤처, SF]","9,001,331"
4,베를린,2013,[액션],"7,166,290"


In [11]:
df.to_csv("/home/lim/dev_ws/eda/ros_eda_project/data/movie_genre_data.csv", sep=',', encoding="utf-8")